# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [29]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [5]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [7]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17464789/17464789 [==============================] - 2s 0us/step
25000 train sequences
25000 test sequences


In [10]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = keras.utils.data_utils.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.utils.data_utils.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [11]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [12]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [14]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10


2022-08-27 19:26:35.023644: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


782/782 [==============================] - 4s 4ms/step - loss: 0.6438 - accuracy: 0.6342 - val_loss: 0.5857 - val_accuracy: 0.6871
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.5350 - accuracy: 0.7317 - val_loss: 0.5252 - val_accuracy: 0.7356
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.4770 - accuracy: 0.7732 - val_loss: 0.4953 - val_accuracy: 0.7565
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.4386 - accuracy: 0.7977 - val_loss: 0.4746 - val_accuracy: 0.7692
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.4116 - accuracy: 0.8136 - val_loss: 0.4663 - val_accuracy: 0.7745
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3934 - accuracy: 0.8217 - val_loss: 0.4522 - val_accuracy: 0.7826
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 0.3799 - accuracy: 0.8300 - val_loss: 0.4513 - val_accuracy: 0.7853
Epoch 8/10
782/78

In [16]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 1s 645us/step - loss: 0.4481 - accuracy: 0.7903
Test score: 0.44813618063926697
Test accuracy: 0.7902799844741821


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [18]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = keras.utils.data_utils.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.utils.data_utils.pad_sequences(x_test, maxlen=maxlen)

In [19]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [20]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [21]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 6s 8ms/step - loss: 0.5969 - accuracy: 0.6781 - val_loss: 0.5077 - val_accuracy: 0.7502
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4532 - accuracy: 0.7884 - val_loss: 0.4549 - val_accuracy: 0.7882
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3901 - accuracy: 0.8269 - val_loss: 0.4629 - val_accuracy: 0.7787
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3483 - accuracy: 0.8492 - val_loss: 0.3948 - val_accuracy: 0.8220
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.3175 - accuracy: 0.8654 - val_loss: 0.3835 - val_accuracy: 0.8296
Epoch 6/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2953 - accuracy: 0.8773 - val_loss: 0.3802 - val_accuracy: 0.8302
Epoch 7/10
782/782 [==============================] - 6s 7ms/step - loss: 0.2793 - accuracy: 0.8847 - val_loss: 0.3696 - val_accuracy: 0.8370
Epoch 

In [23]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = keras.utils.data_utils.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.utils.data_utils.pad_sequences(x_test, maxlen=maxlen)

In [24]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [25]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [26]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.6376 - accuracy: 0.6278 - val_loss: 0.5696 - val_accuracy: 0.7148
Epoch 2/10
782/782 [==============================] - 4s 6ms/step - loss: 0.5063 - accuracy: 0.7588 - val_loss: 0.4856 - val_accuracy: 0.7655
Epoch 3/10
782/782 [==============================] - 4s 6ms/step - loss: 0.4418 - accuracy: 0.7953 - val_loss: 0.4692 - val_accuracy: 0.7826
Epoch 4/10
782/782 [==============================] - 4s 6ms/step - loss: 0.4031 - accuracy: 0.8187 - val_loss: 0.4410 - val_accuracy: 0.7981
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3800 - accuracy: 0.8304 - val_loss: 0.4188 - val_accuracy: 0.8098
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3631 - accuracy: 0.8421 - val_loss: 0.4091 - val_accuracy: 0.8127
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3496 - accuracy: 0.8494 - val_loss: 0.3953 - val_accuracy: 0.8194
Epoch 

In [28]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2764 - accuracy: 0.8859 - val_loss: 0.3648 - val_accuracy: 0.8402
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2738 - accuracy: 0.8870 - val_loss: 0.3700 - val_accuracy: 0.8393
Epoch 3/10
782/782 [==============================] - 4s 6ms/step - loss: 0.2719 - accuracy: 0.8873 - val_loss: 0.3634 - val_accuracy: 0.8402
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2698 - accuracy: 0.8872 - val_loss: 0.3624 - val_accuracy: 0.8417
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2686 - accuracy: 0.8892 - val_loss: 0.3760 - val_accuracy: 0.8389
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2669 - accuracy: 0.8892 - val_loss: 0.3615 - val_accuracy: 0.8417
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2650 - accuracy: 0.8917 - val_loss: 0.3651 - val_accuracy: 0.8415
Epoch 

---
### Machine Learning Foundation (C) 2020 IBM Corporation